In [ ]:
# TODO UPDATE THIS OLD MESSY CODE WITH NEW PANEL

In [1]:
# IMPORT PACKAGES
import pandas as pd
import numpy as np
import itertools
import time
import pickle
from dateutil.relativedelta import relativedelta
from datetime import datetime
from sklearn.decomposition import PCA
from sklearn.utils import resample
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import initializers
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import backend as k
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.callbacks import Callback
import tensorflow as tf
import matplotlib.pyplot as plt

2022-08-14 11:47:26.797746: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
def subsetToAssetUniversePerWeek(temp_df, asset_universe_dict, oos_week):
    # Determine what quarter the oos_week is in
    oos_mnth = pd.to_datetime(oos_week).month
    oos_yr   = pd.to_datetime(oos_week).year
    mnth_qtr = int(np.floor((oos_mnth-1)/3)*3+1)
    if mnth_qtr == 10:
        oos_qtr = str(oos_yr)+'-'+str(mnth_qtr)+'-01'
    else:
        oos_qtr = str(oos_yr)+'-0'+str(mnth_qtr)+'-01'

    # Determine the asset universe
    asset_universe = asset_universe_dict[oos_qtr]
    
    # Subset the training data to the asset universe
    temp_df = temp_df[temp_df.asset.isin(asset_universe)]
    
    return temp_df

In [3]:
def subsetToAssetUniverseFull(df, asset_universe_dict, train_or_test):
    # determine the asset universe to use for whether train or test data
    if train_or_test == 'train':
        index_start = 0
        index_end   = len(asset_universe_dict)-4
    elif train_or_test=='test':
        index_start = len(asset_universe_dict)-4
        index_end   = len(asset_universe_dict)
    else:
        assert(False),('get wit zee program')
        
    # subset to included assets
    for i in range(index_start, index_end):
        # extract this quarter and its included assets
        date = list(asset_universe_dict.keys())[i]
        assets = asset_universe_dict[date]

        # form start and end date for this window
        start_date = datetime.strptime(date, '%Y-%m-%d')
        end_date   = datetime.strptime(date, '%Y-%m-%d') + relativedelta(months=3)

        # drop rows in this time period that are not the included assets
        df = df[~(((df.index>=start_date) & (df.index<end_date)) & (~df.asset.isin(assets)))]
    
    return df

In [4]:
def createPlatonicPortfolioWeights(df, min_weight, lhs_col):
    # sorts assets by return within date and rank
    df = df.sort_values(by=['date', lhs_col])
    df['ranking'] = df.groupby(['date']).cumcount()
    df['counts']  = 1
    df['assets_per_date'] = df.groupby(['date']).counts.sum()
    df['ranking'] = df.ranking/df.assets_per_date

    # determine number of assets per date
    N = df[df.index==np.min(df.index)].shape[0]

    # create min (difference) portfolio weights
    # -about 51% in top half and 49% in bottom half earns 
    #  roughly a 10 basis prem in val data over the naive weights
#     df.loc[df.ranking<0.5, 'y_min']  = 0.49/(N/2)
#     df.loc[df.ranking>=0.5, 'y_min'] = 0.51/(N/2)

    # create lin portfolio weights
    # -determine delta in portfolio weights to increase linearly from min weight, s.t. sum to 1 each week
    # --solves for weight_delta==w_d where min_weight==m_w and N=assets_per_week,
    #   in 1=m_w+(min_w+w_d)+...+(m_w+(N-1)*w_d)
#     df['weight_delta'] = 2*(1-df.assets_per_date*min_weight)/(df.assets_per_date*(df.assets_per_date-1))
#     df['y_lin'] = df.groupby(['date'])['weight_delta'].cumsum()
#     df['y_lin'] = df.y_lin - df.weight_delta + min_weight

    # create exp portfolio weights
#     def softmax(x):
#         e = np.exp(x)
#         return e / np.sum(e)
#     # keep trying a set of weights such that we work
#     #  down to the set where the min weight is just above min_weight
#     for i in range(1,N):
#         y_exp = softmax(np.array(range(N))/N*i)
#         if y_exp[0]<min_weight:
#             break
#     y_exp = softmax(np.array(range(N))/N*(i-1))
#     shift_amount = y_exp[0]-min_weight
#     half_index   = int(len(y_exp)/2)
#     y_exp[:half_index] = y_exp[:half_index]-shift_amount
#     y_exp[half_index:] = y_exp[half_index:]+shift_amount
#     assert(y_exp[0]>=min_weight)
#     num_weeks = len(np.unique(df.index.values))
#     df['y_exp'] = np.tile(y_exp, num_weeks)

    # assign max (difference) portfolio weights
#     df.loc[df.ranking<0.5, 'y_max'] = min_weight
#     df.loc[df.ranking>=0.5, 'y_max'] = (1-min_weight*(N/2))/(N/2)
    
    # create agg(ressive) portfolio weights
#     df['y_agg'] = 0
#     df.loc[df.ranking==np.max(df.ranking), 'y_agg'] = 1
    
    # assign portfolio weights with shorting
    df.loc[df[lhs_col] < 0, 'assets_neg_per_date'] = 1
    df['assets_neg_per_date'] = df.groupby('date').assets_neg_per_date.transform('sum')
    df.loc[(df.assets_neg_per_date == 1) & (df.ranking==np.min(df.ranking)), 'y_short'] = -1
    df.loc[(df.assets_neg_per_date == 1) & (df.ranking==np.min(df.ranking)), 'y_short'] = -1
    df.loc[(df.assets_neg_per_date == 1) & (df.ranking==np.max(df.ranking)), 'y_short'] = 1
    df.loc[(df.assets_neg_per_date == 1) & (df.ranking==np.min(df.ranking)) 
                                         & (df.r_tplus1>-0.005), 'y_short'] = 0
    df.loc[(df.assets_neg_per_date == 1) & (df.ranking==np.max(df.ranking)) 
                                         & (df.r_tplus1<0.005), 'y_short'] = 0
    df.loc[(df.assets_neg_per_date==2), 'y_short'] = -1
    df.loc[(df.assets_neg_per_date==2)&(df.r_tplus1>-.01)&(df.y_short==-1), 'y_short'] = 0
    df.loc[(df.assets_neg_per_date == 0), 'y_short'] = 1
    df.loc[(df.assets_neg_per_date == 0) & (df.r_tplus1<0.005), 'y_short'] = 0
    df = df.drop('assets_neg_per_date', axis=1)
    
    # confirm weights are equal to one each date for each type of LHS var
#     for lhs_y in ['y_min', 'y_lin', 'y_exp', 'y_max', 'y_agg']:
#         assert(len(np.unique(df.index)) == 
#                np.sum(np.isclose(df.groupby(['date'])[lhs_y].sum(), 1,
#                                  rtol=1e-2, atol=1e-2)))
    
    # clean up columns
    #df = df.drop(['ranking', 'counts', 'assets_per_week', 'weight_delta'],
    #              axis=1)
    df = df.drop(['ranking', 'counts', 'assets_per_date'], axis=1)

    return df

In [5]:
def flattenToOneObPerDate(df, lhs_col):
    # initialize df to return
    final_df = pd.DataFrame(data={'date': np.unique(df.index)})
    final_df = final_df.set_index('date')

    # for each asset, update column names
    assets = list(np.unique(df.asset))
    for asset in assets:
        # subset to relavant rows
        temp_df = df[df.asset == asset]

        # drop asset column and macro columns
        temp_df = temp_df.drop('asset', axis=1)
        macro_cols = list(temp_df.filter(regex='macro_', axis=1).columns.values)
        temp_df = temp_df.drop(macro_cols, axis=1)

        # form lhs variables
        temp_df = temp_df.rename(columns={'y_short': 'y_short_'+asset,
                                          lhs_col: lhs_col+'_'+asset})

        # clean up covar columns
        covar_cols = list(temp_df.filter(regex='covar_', axis=1).columns.values)
        for covar_col in covar_cols:
            temp_df = temp_df.rename(columns={covar_col: covar_col+'_'+asset})
        final_df = final_df.merge(temp_df,
                                  on='date',
                                  how='inner',
                                  validate='one_to_one')

    # merge on macro columns to final df
    macro_cols = list(df.filter(regex='macro_', axis=1).columns.values)
    macro_df = df[macro_cols]
    macro_df = macro_df.drop_duplicates()
    final_df = final_df.merge(macro_df,
                              on='date',
                              how='inner',
                              validate='one_to_one')

    # reset column order
    all_cols      = list(final_df.columns.values)
    y_cols        = [col for col in all_cols if col[:2] == 'y_']
    r_tplus_cols  = [col for col in all_cols if col[:8] == lhs_col]
    covar_cols    = [col for col in all_cols if col[:6] == 'covar_']
    final_df      = final_df[y_cols+r_tplus_cols+macro_cols+covar_cols]

    return final_df

In [6]:
def customLoss(lmbd):
    def loss(y_true, y_pred):
        naive_weights    = tf.repeat(0.05, 20) # note: manually programmed 20 assets
        weeks_neg_return = -tf.tensordot(y_true, tf.transpose(y_pred), axes=1)
        dist_from_naive  = tf.math.reduce_sum(tf.square(naive_weights - y_pred))
        return float(weeks_neg_return) + lmbd*float(dist_from_naive)
    
    return loss

In [7]:
def buildFFNN(i, train_rhs, train_lhs,
              first_layer_width, leaky_relu, 
              l1_penalty, apply_dropout, dropout_rate,
              number_hidden_layer, learning_rate,
              loss, lmbd,
              weight_initializer, bias_initializer):

    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input(shape=(train_rhs.shape[1],)))
    model.add(Dense(first_layer_width, activation='gelu',
                    kernel_regularizer=regularizers.l1(l1=l1_penalty),
                    kernel_initializer=weight_initializer,
                    bias_initializer=bias_initializer))
    model.add(Dropout(dropout_rate, seed=i)) 
    model.add(BatchNormalization())
        
    if number_hidden_layer >= 2:
        model.add(Dense(64, activation='gelu',
                        kernel_regularizer=regularizers.l1(l1=l1_penalty),
                        kernel_initializer=weight_initializer,
                        bias_initializer=bias_initializer))
        model.add(Dropout(dropout_rate, seed=i))
        model.add(BatchNormalization())
         
            
    if number_hidden_layer >= 3:
        model.add(Dense(32, activation='gelu',
                        kernel_regularizer=regularizers.l1(l1=l1_penalty),
                        kernel_initializer=weight_initializer,
                        bias_initializer=bias_initializer))
        model.add(Dropout(dropout_rate, seed=i)) 
        model.add(BatchNormalization())
        
            
    if number_hidden_layer >= 4:
        model.add(Dense(16, activation='gelu',
                        kernel_regularizer=regularizers.l1(l1=l1_penalty),
                        kernel_initializer=weight_initializer,
                        bias_initializer=bias_initializer))
        model.add(Dropout(dropout_rate, seed=i)) 
        model.add(BatchNormalization())
        
            
    if number_hidden_layer == 5:
        model.add(Dense(8, activation='gelu',
                        kernel_regularizer=regularizers.l1(l1=l1_penalty),
                        kernel_initializer=weight_initializer,
                        bias_initializer=bias_initializer))
        model.add(Dropout(dropout_rate, seed=i)) 
        model.add(BatchNormalization())
        
            
    model.add(Dense(train_lhs.shape[1], activation="linear"))
    #model.add(Softmax())
    
    # Compile the model
    if loss=='mse':
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                      loss='mse', metrics=['mse', 'mae'])
    if loss=='custom':
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                      loss=customLoss(lmbd=lmbd))

    return model

In [8]:
# TODO: FINISH THIS SKETCHED OUT CODE
# TODO: CHECK I USE ALL ARGUMENTS
def buildTransformer(i, train_rhs, train_lhs,
                     first_layer_width, l1_penalty, dropout_rate,
                     number_hidden_layer, learning_rate, loss, lmbd,
                     weight_initializer, bias_initializer,
                     dim_k, num_heads):

    # set up input
    input_dim  = train_rhs.shape[1]
    output_dim = train_lhs.shape[1]
    input_vec  = tf.keras.Input(shape=input_dim)

    # build Q, K, V
    q   = Dense(dim_k, activation='linear',
                kernel_initializer=weight_initializer,
                use_bias=False)(input_vec)
    k   = Dense(dim_k, activation='linear',
                kernel_initializer=weight_initializer,
                use_bias=False)(input_vec)
    v   = Dense(dim_k, activation='linear',
                kernel_initializer=weight_initializer,
                use_bias=False)(input_vec)

    # build multi-head attention
    attn = MultiHeadAttention(num_heads=num_heads,
                              dropout=dropout_rate,
                              output_shape=input_dim,
                              kernel_initializer=weight_initializer,
                              bias_initializer=bias_initializer,
                              kernel_regularizer=regularizers.l1(l1=l1_penalty))(q=q, k=k, v=v)
    attn = Dropout(dropout_rate, seed=i)(attn)
    attn = input_vec + attn
    attn = BatchNormalization()(attn)

    # build ffnn
    x  = Dense(first_layer_width, activation='gelu',
              kernel_regularizer=regularizers.l1(l1=l1_penalty),
              kernel_initializer=weight_initializer,
              bias_initializer=bias_initializer)(x)
    x  = Dropout(dropout_rate, seed=i)(x)
    x  = BatchNormalization()(x)

    if number_hidden_layer >= 2:
        x = Dense(64, activation='gelu',
                  kernel_regularizer=regularizers.l1(l1=l1_penalty),
                  kernel_initializer=weight_initializer,
                  bias_initializer=bias_initializer)(x)
        x = Dropout(dropout_rate, seed=i)(x)
        x = BatchNormalization()(x)

    if number_hidden_layer >= 3:
        x = Dense(32, activation='gelu',
                  kernel_regularizer=regularizers.l1(l1=l1_penalty),
                  kernel_initializer=weight_initializer,
                  bias_initializer=bias_initializer)(x)
        x = Dropout(dropout_rate, seed=i)(x)
        x = BatchNormalization()(x)

    if number_hidden_layer >= 4:
        x = Dense(16, activation='gelu',
                  kernel_regularizer=regularizers.l1(l1=l1_penalty),
                  kernel_initializer=weight_initializer,
                  bias_initializer=bias_initializer)(x)
        x = Dropout(dropout_rate, seed=i)(x)
        x = BatchNormalization()(x)

    if number_hidden_layer == 5:
        x = Dense(8, activation='gelu',
                  kernel_regularizer=regularizers.l1(l1=l1_penalty),
                  kernel_initializer=weight_initializer,
                  bias_initializer=bias_initializer)(x)
        x = Dropout(dropout_rate, seed=i)(x)
        x = BatchNormalization()(x)

    # form final layer to map to weights
    x = Dense(output_dim, activation='linear')(x)
    output_vec = Softmax()(x)        
    model = Model(input_vec, output_vec)

    # Compile the model
    if loss=='mse':
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                      loss='mse', metrics=['mse', 'mae'])
    if loss=='custom':
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                      loss=customLoss(lmbd=lmbd))

    return model

In [9]:
def fitNN(train_df, hps_yhats_dict, lhs_col, val_df=None):
    # Extract the hyperparameters
    first_layer_width   = hps_yhats_dict['first_layer_width']
    number_hidden_layer = hps_yhats_dict['number_hidden_layer']
    learning_rate       = hps_yhats_dict['learning_rate']
    batch_size          = hps_yhats_dict['batch_size']
    l1_penalty          = hps_yhats_dict['l1_penalty']
    number_ensemble     = hps_yhats_dict['number_ensemble']
    epoch               = hps_yhats_dict['epoch']
    early_stopping      = hps_yhats_dict['early_stopping']
    bootstrap_pct       = hps_yhats_dict['bootstrap_pct']
    dropout_rate        = hps_yhats_dict['dropout_rate']
    apply_dropout       = hps_yhats_dict['apply_dropout']
    leaky_relu          = hps_yhats_dict['leaky_relu']
    patience            = hps_yhats_dict['patience']
    loss                = hps_yhats_dict['loss']
    lmbd                = hps_yhats_dict['lmbd']
    bias_initial        = hps_yhats_dict['bias_initial']
    weight_initial      = hps_yhats_dict['weight_initial']
    target_mse          = hps_yhats_dict['target_mse']

    # Obtain the lhs and rhs column names
    all_cols      = list(train_df.columns.values)
    y_cols        = [col for col in all_cols if col[:2] == 'y_']
    r_tplus_cols  = [col for col in all_cols if col[:8] == lhs_col]
    covar_cols    = [col for col in all_cols if col[:6] == 'covar_']
    macro_cols    = [col for col in all_cols if col[:6] == 'macro_']
    rhs_cols      = covar_cols+macro_cols
    if loss=='mse':
        lhs_cols = y_cols
    if loss=='custom':
        lhs_cols = r_tplus_cols
        
    # Loop over the ensembles to a build model for each
    assert(number_ensemble <= 5)
    models = []
    for i in range(number_ensemble):
        # Bootstrap the rows so different models in the ensemble are less correlated
        model_train_df = resample(train_df, replace=False, 
                                  n_samples=int(train_df.shape[0]*bootstrap_pct),
                                  random_state=i)

        # Obtain the training input and output data and, if passed, validation data
        train_rhs   = model_train_df[rhs_cols].values
        train_lhs   = model_train_df[lhs_cols].values
        if val_df is not None:
            val_rhs = val_df[rhs_cols].values  
            val_lhs = val_df[lhs_cols].values

        # Initialize parameters, but with different values across ensembled models
        if bias_initial=='constant':
            bias_initializer=initializers.Constant(0.01)
        if bias_initial=='he':
            bias_initializer=initializers.HeNormal(seed=i)
        if bias_initial=='glorot':
            bias_initializer=initializers.GlorotUniform(seed=i)
        if bias_initial=='uniform':
            bias_initializer=initializers.RandomUniform(seed=i)
        if weight_initial=='he':
            weight_initializer=initializers.HeNormal(seed=i)
        if weight_initial=='glorot':
            weight_initializer=initializers.GlorotUniform(seed=i)
        if weight_initial=='uniform':
            weight_initializer=initializers.RandomUniform(seed=i)
        
        # Build the model
        model = buildFFNN(i, train_rhs, train_lhs,
                          first_layer_width, leaky_relu, 
                          l1_penalty, apply_dropout, dropout_rate,
                          number_hidden_layer, learning_rate,
                          loss, lmbd,
                          weight_initializer, bias_initializer)
        
        # Set up early stopping
        es = EarlyStopping(monitor='mse', mode='min', verbose=1, 
                           patience=patience, min_delta=target_mse*0.001)

        # Fit the model
        with tf.device('/CPU:0'):
            model.fit(x=train_rhs, y=train_lhs, 
                      batch_size=batch_size,
                      epochs=epoch, verbose=1,
                      workers=10, callbacks=[es])
           
        # Save the model
        models.append(model)
        print('\n')

    return models

In [10]:
def genYhats(oos_df, models, lhs_col):
    # Obtain the lhs and rhs column names
    all_cols      = list(oos_df.columns.values)
    y_cols        = [col for col in all_cols if col[:2] == 'y_']
    r_tplus_cols  = [col for col in all_cols if col[:8] == lhs_col]
    covar_cols    = [col for col in all_cols if col[:6] == 'covar_']
    macro_cols    = [col for col in all_cols if col[:6] == 'macro_']
    rhs_cols      = covar_cols+macro_cols
    
    # Obtain the RHS data
    oos_rhs   = oos_df[rhs_cols].values  
    oos_lhs   = oos_df[y_cols].values
    
    # Form the yhats
    yhats = []
    for model in models:
        with tf.device('/CPU:0'):
            yhats.append(model.predict(oos_rhs))
    yhats = np.mean(yhats, axis=0)

    # Return results
    return pd.DataFrame(data={'date': oos_df.index.values[0],
                              lhs_col: list(oos_df[r_tplus_cols].values),
                              'ys':    list(oos_lhs),
                              'yhats': list(yhats)})

In [11]:
def runCV(df, asset_universe_dict, lhs_col,
          last_train_year=2018, val_end_year=2020, arch_name=None):
    # Initialize hp result objects
    results_list    = []
    hps_mse_df_list = []
    
    # Initialize the hyperparameter grid  
    first_layer_widths   = [256] 
    number_hidden_layers = [5]
    learning_rates       = [1e-4] # TODO NARROW TO CLOSER RANGE
    batch_sizes          = [128]  # explore once arch is dialed in [64, 128, 256]
    l1_penalties         = [0.001] # Doesnt seem to do much; maybe explore once dialed in  [0.001, 0.01, 0.1, 1]
    number_ensembles     = [3] # Explore cranking up once dialed in, 7, 9 or way more
    epochs               = [200] # explore cranking it up once more dialed in [100, 200, 500]
    # Explore cutting down once dialed in as never seem to use more than 100
    # or figure out a way to turn learning way down so we can turn epoch up
    early_stoppings      = [True] 
    bootstrap_pcts       = [0.99] # explore more once arch is dialed in [0.9, 0.95, 0.99]
    dropout_rates        = [0.1] # explore more once arch is dialed in [0.01, 0.1, 0.2, 0.3, 0.4]
    apply_dropouts       = [True] 
    leaky_relus          = [0]          # explore once arch is dialed in [0, 0.01, 0.1]
    patiences            = [16]          # explore more once  arch is dialed in [20, 50, 100]
    losses               = ['mse']      # explore custom once arch is dialed in
    lambdas              = [0]          # explore custom once arch is dialed in [0, 0.01, 0.1]
    bias_initials        = ['he']       # once architecture is dialed, circle back to playing with these
    weight_initials      = ['uniform']  # once architecture is dialed, circle back to playing with these
    lhs_vars             = ['short']      # once architecture is dialed, circle back to playing with these
    
    # Determine the dates in the validation window  
    val_dates = np.unique(df[(df.index.year > last_train_year)  
                             & (df.index.year <= val_end_year)].index.values)  

    # Generate yhats for every hyperparameter grid point
    for hps in itertools.product(first_layer_widths,
                                 number_hidden_layers,
                                 learning_rates,
                                 batch_sizes,
                                 l1_penalties,
                                 number_ensembles,
                                 epochs,
                                 early_stoppings,
                                 bootstrap_pcts,
                                 dropout_rates,
                                 apply_dropouts,
                                 leaky_relus,
                                 patiences,
                                 losses,
                                 lambdas,
                                 bias_initials,
                                 weight_initials,
                                 lhs_vars):

        # don't repeat all weighted custom loss functions when loss function is mse
        if (hps[13] == 'mse') & (hps[14] > 0):
            continue
            
        # build the hp and yhats dictionary
        hps_yhats_dict = {'first_layer_width': hps[0],
                          'number_hidden_layer': hps[1],
                          'learning_rate': hps[2],
                          'batch_size': hps[3],
                          'l1_penalty': hps[4],
                          'number_ensemble': hps[5],
                          'epoch': hps[6],
                          'early_stopping': hps[7],
                          'bootstrap_pct': hps[8],
                          'dropout_rate': hps[9],
                          'apply_dropout': hps[10],
                          'leaky_relu': hps[11],
                          'patience': hps[12],
                          'loss': hps[13],
                          'lmbd': hps[14],
                          'bias_initial': hps[15],
                          'weight_initial': hps[16],
                          'lhs_var': hps[17],
                          'results_df': pd.DataFrame()}
        
        # determine relevant lhs variables and other column names to use
        all_cols      = list(df.columns.values)
        y_cols        = [col for col in all_cols if col[:(len(hps_yhats_dict['lhs_var'])+3)] == 
                         ('y_'+hps_yhats_dict['lhs_var']+'_')]
        r_tplus_cols  = [col for col in all_cols if col[:8] == lhs_col]
        covar_cols    = [col for col in all_cols if col[:6] == 'covar_']
        macro_cols    = [col for col in all_cols if col[:6] == 'macro_']
        
        # add target mse to validation results
        naive_forecast = np.mean(df[df.index.year>last_train_year][y_cols].values)
        target_mse = np.mean(np.square(df[df.index.year>last_train_year][y_cols].values-naive_forecast))
        hps_yhats_dict['target_mse'] = target_mse
        
        # fit on all val dates
        print(hps_yhats_dict, '\n')
        tic = time.perf_counter()
        training_model_mses = np.array([])
        val_model_mses      = np.array([])
        for val_date in val_dates:
            # remove irrelevant lhs varibles
            temp_df  = df[y_cols+r_tplus_cols+covar_cols+macro_cols].copy()
            
            # Form train and validation data frames
            temp_df  = temp_df[df.index <= val_date]
            train_df = temp_df[temp_df.index < val_date]
            val_df   = temp_df[temp_df.index == val_date]
            
            # Fit model and generate yhats for val week
            models         = fitNN(train_df, hps_yhats_dict, lhs_col)
            val_results_df = genYhats(val_df, models, lhs_col)

            # Append this date's results
            hps_yhats_dict['results_df'] = pd.concat((hps_yhats_dict['results_df'], val_results_df))

            # Display Val Date results
            # train data results just to check
            train_results_df = genYhats(train_df, models, lhs_col)
            train_ys         = train_results_df.ys.values[0]
            train_yhats      = train_results_df.yhats.values[0]
            train_naive      = np.repeat(np.mean(train_ys), len(train_ys))
            train_mse_naive  = np.mean(np.square(train_ys-train_naive))
            train_mse_model  = np.mean(np.square(train_ys-train_yhats))
            print('Train mse Naive: '+str(np.round(train_mse_naive, 6)))
            print('Train mse Model: '+str(np.round(train_mse_model, 6)))
            training_model_mses = np.concatenate((training_model_mses, 
                                                  np.array([train_mse_model]))) # save to report after all val weeks predicted
            
            # mse results
            print('\n')
            print(val_date)
            val_ys        = val_results_df.ys.values[0]
            val_yhats     = val_results_df.yhats.values[0]
            
            print(val_ys)
            print(val_yhats)
            val_naive     = np.repeat(np.mean(val_ys), len(val_ys))
            val_mse_naive = np.mean(np.square(val_ys-val_naive))
            val_mse_model = np.mean(np.square(val_ys-val_yhats))
            val_model_mses = np.concatenate((val_model_mses,
                                             np.array([val_mse_model]))) # save to report after all val weeks predicted
            print('Val date mse Naive: '+str(np.round(val_mse_naive, 6)))
            print('Val date mse Model: '+str(np.round(val_mse_model, 6)))
            
            #yhat distribution results
            print('Distribution of val week y hats:')
            print('Max : '+str(np.round(np.max(val_yhats), 2)))
            print('Median: '+str(np.round(np.median(val_yhats), 2)))
            print('Min : '+str(np.round(np.min(val_yhats), 2)))

            # actual return results
            val_rets      = val_df[r_tplus_cols].values
            val_ret_naive = np.mean(val_rets)
            val_ret_model = np.sum(val_yhats*val_rets)
            print('Naive val return: '+str(np.round(val_ret_naive, 8)))
            print('Model val return: '+str(np.round(val_ret_model, 8)))

            # Display to date results
            # mse results
            print('\nEntire val period results:')
            oos_ys        = np.vstack(hps_yhats_dict['results_df'].ys.values)
            oos_yhats     = np.vstack(hps_yhats_dict['results_df'].yhats.values)
            oos_naive     = np.repeat(np.mean(oos_ys), oos_ys.shape[1]*oos_ys.shape[0]).reshape(-1, oos_ys.shape[1])
            oos_mse_naive = np.mean(np.square(oos_ys-oos_naive))
            oos_mse_model = np.mean(np.square(oos_ys-oos_yhats))
            print('MSE Naive: '+str(np.round(oos_mse_naive, 6)))
            print('MSE Model: '+str(np.round(oos_mse_model, 6)))
            if oos_mse_model < oos_mse_naive:
                print('Winning MSE?: Yes! ' + str(np.round(oos_mse_naive-oos_mse_model, 4)))
            else:
                print('Winning MSE?: No! ' + str(np.round(oos_mse_naive-oos_mse_model, 4)))
                
            #yhat distribution results
            print('Distribution of val period y hats:')
            print('Max : '+str(np.round(np.max(oos_yhats), 2)))
            print('Median: '+str(np.round(np.median(oos_yhats), 2)))
            print('Min : '+str(np.round(np.min(oos_yhats), 2)))

            # actual return results
            num_dates     = hps_yhats_dict['results_df'].shape[0]
            oos_rets      = np.vstack(hps_yhats_dict['results_df'][lhs_col].values)
            oos_ret_naive = np.prod(1+np.mean(oos_rets, axis=1))**(1/num_dates)-1
            oos_ret_model = np.prod(1+np.sum(oos_rets*oos_yhats, axis=1))**(1/num_dates)-1
            print('Naive avg per date return to date: '+str(np.round(oos_ret_naive, 4)))
            print('Model avg per date return to date: '+str(np.round(oos_ret_model, 4)))
            if oos_ret_model > oos_ret_naive:
                print('Winning return?: Yes! ' + str(np.round(oos_ret_model-oos_ret_naive, 8)))
            else:
                print('Winning return?: No! ' + str(np.round(oos_ret_model-oos_ret_naive, 8)))
            
            # Stop running if results are not good enough
#             month = val_date.astype('datetime64[M]').astype(int)%12+1
#             if (month >= 2) & ((oos_ret_model-oos_ret_naive)<-0.003):
#                 break   
#             if (month >= 3) & ((oos_ret_model-oos_ret_naive)<-0.001):
#                 break  
#             if (month >= 6) & ((oos_ret_model-oos_ret_naive)<0):
#                 break 
#             year = val_date.astype('datetime64[Y]').astype(int)+1970
#             if (year >= 2019) & ((oos_ret_model-oos_ret_naive)<0):
#                 break
                
            # space out the results
            print('\n')

        # Save run time and space out result print out
        toc = time.perf_counter()
        print('\n\n\n')

        # Update this HP point's results and append to list of results
        hps_yhats_dict['end_date']      = val_date
        hps_yhats_dict['mse']           = oos_mse_model
        hps_yhats_dict['val_ret_delta'] = oos_ret_model-oos_ret_naive
        results_list.append(hps_yhats_dict)

        # Save the ongoing results as a csv to be able to watch
        cv_results_dict = hps_yhats_dict.copy()
        del cv_results_dict['results_df']
        cv_results_dict['runtime_mins']     = round((toc - tic)/60, 0)  
        cv_results_dict['num_lhs_vars']     = len(r_tplus_cols)
        cv_results_dict['num_rhs_vars']     = len(covar_cols+macro_cols)
        cv_results_dict['arch_name']        = arch_name
        cv_results_dict['first_val_year']   = str(last_train_year+1) 
        cv_results_dict['first_train_year'] = str(np.min(df.index.year))
        cv_results_dict['models_avg_training_mses'] = np.mean(training_model_mses)
        cv_results_dict['models_avg_diff_btwn_train_and_val_mse'] = np.mean(training_model_mses-val_model_mses)
        # report metrics by year if relevant
        if last_train_year == 2017:
            temp_df   = hps_yhats_dict['results_df'].copy()
            rets      = np.vstack(temp_df[temp_df.date.dt.year==2018][lhs_col].values)
            ys        = np.vstack(temp_df[temp_df.date.dt.year==2018].ys.values)
            yhats     = np.vstack(temp_df[temp_df.date.dt.year==2018].yhats.values)
            num_weeks = temp_df.shape[0]
            cv_results_dict['ret_naive_2018'] = np.prod(1+np.mean(rets, axis=1))**(1/num_weeks)-1
            cv_results_dict['ret_model_2018'] = np.prod(1+np.sum(rets*yhats, axis=1))**(1/num_weeks)-1
            cv_results_dict['ret_diff_2018']  = np.prod((1+np.sum(rets*yhats, axis=1))-(1+np.mean(rets, axis=1)))**(1/num_weeks)-1
            cv_results_dict['ret_btc_2018']   = np.prod(1+df[df.index.year==2018][lhs_col+'_bitcoin'].values)**(1/num_weeks)-1
            cv_results_dict['ret_eth_2018']   = np.prod(1+df[df.index.year==2018][lhs_col+'_ethereum'].values)**(1/num_weeks)-1
            cv_results_dict['mse_model_2018'] = np.mean(np.square(ys-yhats))
            cv_results_dict['mse_model_divided_naive_2018'] = np.mean(np.square(ys-yhats))/target_mse
        
        if last_train_year <= 2018:
            temp_df   = hps_yhats_dict['results_df'].copy()
            rets      = np.vstack(temp_df[temp_df.date.dt.year==2019][lhs_col].values)
            ys        = np.vstack(temp_df[temp_df.date.dt.year==2019].ys.values)
            yhats     = np.vstack(temp_df[temp_df.date.dt.year==2019].yhats.values)
            num_weeks = temp_df.shape[0]
            cv_results_dict['ret_naive_2019'] = np.prod(1+np.mean(rets, axis=1))**(1/num_weeks)-1
            cv_results_dict['ret_model_2019'] = np.prod(1+np.sum(rets*yhats, axis=1))**(1/num_weeks)-1
            cv_results_dict['ret_btc_2019']   = np.prod(1+df[df.index.year==2019][lhs_col+'_bitcoin'].values)**(1/num_weeks)-1
            cv_results_dict['ret_eth_2019']   = np.prod(1+df[df.index.year==2019][lhs_col+'_ethereum'].values)**(1/num_weeks)-1
            cv_results_dict['mse_model_2019'] = np.mean(np.square(ys-yhats))
            cv_results_dict['mse_model_divided_naive_2019'] = np.mean(np.square(ys-yhats))/target_mse
        
        if last_train_year <= 2019:
            temp_df   = hps_yhats_dict['results_df'].copy()
            rets      = np.vstack(temp_df[temp_df.date.dt.year==2020][lhs_col].values)
            ys        = np.vstack(temp_df[temp_df.date.dt.year==2020].ys.values)
            yhats     = np.vstack(temp_df[temp_df.date.dt.year==2020].yhats.values)
            num_weeks = temp_df.shape[0]
            cv_results_dict['ret_naive_2020'] = np.prod(1+np.mean(rets, axis=1))**(1/num_weeks)-1
            cv_results_dict['ret_model_2020'] = np.prod(1+np.sum(rets*yhats, axis=1))**(1/num_weeks)-1
            cv_results_dict['ret_btc_2020']   = np.prod(1+df[df.index.year==2020][lhs_col+'_bitcoin'].values)**(1/num_weeks)-1
            cv_results_dict['ret_eth_2020']   = np.prod(1+df[df.index.year==2020][lhs_col+'_ethereum'].values)**(1/num_weeks)-1
            cv_results_dict['mse_model_2020'] = np.mean(np.square(ys-yhats))
            cv_results_dict['mse_model_divided_naive_2020'] = np.mean(np.square(ys-yhats))/target_mse
        
        hps_mse_df_list.append(pd.DataFrame(cv_results_dict, index=[0]))
        cv_df = pd.concat(hps_mse_df_list, ignore_index=True)
        
        timestr = time.strftime("%Y%m%d_%H%M%S")
        fp = '../4-output/cv-results-custom-' + timestr + '.csv'
        cv_df.to_csv(fp, index=False)

    return results_list

In [12]:
def labelPortfolioWeights(df):
    # set parameters for tertile weights
    # note: can't go to zero on any as there may be week where that is only yhat
    bottom_tertile = 1/6
    mid_tertile = 1/3
    top_tertile = (1-bottom_tertile-mid_tertile)

    # create portfolio weights
    df = df.sort_values(by=['date', 'yhat'])
    df['counts']  = 1
    df['total_assets_per_week_tertile'] = df.groupby(['date', 'yhat']).counts.transform('sum')
    df.loc[df.yhat == -1, 'prtfl_wght'] = bottom_tertile / df[df.yhat == -1].total_assets_per_week_tertile
    df.loc[df.yhat == 0,  'prtfl_wght'] = mid_tertile / df[df.yhat == 0].total_assets_per_week_tertile
    df.loc[df.yhat == 1,  'prtfl_wght'] = top_tertile / df[df.yhat == 1].total_assets_per_week_tertile

    # clean up
    df = df.drop(['counts', 'total_assets_per_week_tertile'], axis=1)

    # fix weeks where portfolio weights add up to less than 1
    temp_df = df.groupby(['date'])[['prtfl_wght']].sum()
    temp_df = temp_df[~np.isclose(temp_df.prtfl_wght, 1)]
    if 0<temp_df.shape[0]:
        for i in range(temp_df.shape[0]):
            date = temp_df.index.values[i]
            total_weight = temp_df.prtfl_wght.values[i]
            df.loc[df.index == date, 'prtfl_wght'] = df[df.index==date].prtfl_wght * (1/total_weight)
            
    # confirm portfolio weights roughly sum to 1 for each week
    assert(len(np.unique(df.index)) == 
           np.sum(np.isclose(df.groupby(['date']).prtfl_wght.sum(), 1,
                             rtol=1e-2, atol=1e-2)))

    return df


In [13]:
def calcAnnualTransactionCosts(df):
    # merge on the previous week's holdings to the new holdings
    temp_df = df.copy()
    temp_df = temp_df[temp_df.index<np.max(temp_df.index)]
    temp_df.index = temp_df.index+np.timedelta64(1, 'D')
    temp_df = temp_df[['asset', 'prtfl_wght']]
    temp_df = temp_df.rename(columns={'prtfl_wght': 'prtfl_wght_tm7'})
    df      = df.merge(temp_df,
                       on=['date', 'asset'],
                       how='outer',
                       validate='one_to_one')

    # calc weekly turnover and ensure it has the appropriate range
    df['asset_to'] = np.abs(df.prtfl_wght - df.prtfl_wght_tm7)
    to_df = df.groupby('date')[['asset_to']].sum().reset_index()
    assert((np.min(to_df.asset_to)>=0) & (np.max(to_df.asset_to<=2)))

    # correct the first and last week valid for buying the initial port and liquidating
    to_df.loc[0, 'asset_to'] = 1
    to_df = pd.concat((to_df, pd.DataFrame(data={'date': np.max(temp_df.index.values)+np.timedelta64(1, 'D'),
                                                 'asset_to': 1}, index=[0])))
    to_df = to_df.reset_index(drop=True)

    # add transaction costs assuming maker and taker fee of 20 bps each
    to_df['tc'] = to_df.asset_to*0.002

    # return annualize transaction cost
    return -np.sum(to_df.tc)/(to_df.shape[0]/52)

In [14]:
def calcPortfolioReturn(r_df):
    num_days = r_df.shape[0]
    if np.sum(r_df.r.values <= -1)>=1:
        return -1
    else:
        tot_ret   = np.product(r_df.r.values+1)-1
        daily_ret = (tot_ret+1)**(1/num_days)-1
        annl_ret  = (daily_ret+1)**(365.25)-1
        return annl_ret

In [15]:
def calcPortfolioSharpe(df):
    daily_sharpe = np.mean(df.r.values)/np.std(df.r.values)
    annl_sharpe  = daily_sharpe*np.sqrt(365.25)
    return annl_sharpe

In [16]:
def max_draw_down(df):
    cumulative_ret=(df.r+1).cumprod()
    roll_max=cumulative_ret.rolling(len(cumulative_ret), min_periods=1).max()
    daily_drawdown=cumulative_ret/roll_max
    max_daily_drawdown=daily_drawdown.min() - 1
    return max_daily_drawdown

In [17]:
def max_1_week_loss(df):
    max_loss=(df['r']+1).rolling(7).apply(np.prod)
    max_loss_minus=max_loss.min()-1
    return max_loss_minus

In [18]:
def findPsuedoOOSWeek(train_df, oos_df):
    # obtain rhs values for oos week and all training data
    all_cols         = list(oos_df.columns.values)
    rhs_cols         = [col for col in all_cols if col[:6] == 'covar_']
    oos_rhs_values   = oos_df[rhs_cols].values
    train_rhs_values = train_df[rhs_cols].values
    
    # find the index of training data that most resembles the oos week's rhs
    # TO DO: Make this better by taking it within cycle week
    weekly_l2_norm   = np.sqrt(np.sum(np.square(train_rhs_values - oos_rhs_values), axis=1))
    matching_week    = np.argmin(weekly_l2_norm)

    # return that week of training data as pusedo oos week
    return train_df[matching_week:(matching_week+1)]

In [19]:
def GenTestYhats(df, lhs_col, opt_hps, test_year=2022): 
    test_dates = np.unique(df[df.index.year == test_year].index.values)

    test_df = pd.DataFrame()

    for test_date in test_dates:
        print(test_date, '\n')
        temp_df       = df[df.index <= test_date].copy()
        train_df      = temp_df[temp_df.index < test_date].copy()
        oos_df        = temp_df[temp_df.index == test_date].copy()

        models = fitNN(train_df, opt_hps, lhs_col)

        oos_results_df = genYhats(oos_df, models, lhs_col)

        test_df = pd.concat((test_df, oos_results_df))

        # oos date results
        oos_ys        = oos_results_df.ys.values[0]
        oos_yhats     = oos_results_df.yhats.values[0]
        oos_rets      = np.vstack(oos_results_df[lhs_col].values)
        oos_ret_naive = np.mean(oos_rets)
        oos_ret_model = np.prod(1+np.sum(oos_yhats*oos_rets, axis=1))**(1/test_df.shape[0])-1
        print('OOS week results:')
        print('Naive oos return: '+str(np.round(oos_ret_naive, 8)))
        print('Model oos return: '+str(np.round(oos_ret_model, 8)))
        print('\n')

        # oos results
        oos_ys    = np.vstack(test_df.ys.values)
        oos_yhats = np.vstack(test_df.yhats.values)
        oos_naive = np.repeat(np.mean(oos_ys), oos_ys.shape[1]*oos_ys.shape[0]).reshape(-1, oos_ys.shape[1])
        oos_mse_naive = np.mean(np.square(oos_ys-oos_naive))
        oos_mse_model = np.mean(np.square(oos_ys-oos_yhats))
        print('full oos period results:')
        print('oos mse naive: '+str(np.round(oos_mse_naive, 6)))
        print('oos mse model: '+str(np.round(oos_mse_model, 6)))
        print('Distribution of oos y hats:')
        print('Max : '+str(np.round(np.max(oos_yhats), 2)))
        print('Median: '+str(np.round(np.median(oos_yhats), 2)))
        print('Min : '+str(np.round(np.min(oos_yhats), 2)))
        oos_rets      = np.vstack(test_df[lhs_col].values)
        oos_ret_naive = np.prod(np.mean(oos_rets,axis=1)+1)**(1/test_df.shape[0])-1
        oos_ret_model = np.prod(1+np.sum(oos_yhats*oos_rets, axis=1))**(1/test_df.shape[0])-1
        print('Naive oos return: '+str(np.round(oos_ret_naive, 8)))
        print('Model oos return: '+str(np.round(oos_ret_model, 8)))
        if oos_ret_model > oos_ret_naive:
            print('Winning return?: Yes! ' + str(np.round(oos_ret_model-oos_ret_naive, 8)))
        else:
            print('Winning return?: No! ' + str(np.round(oos_ret_model-oos_ret_naive, 8)))
        print('\n')
    
    return test_df


In [ ]:
# RUN THE CODE

# Load in the data
#input_fp = '../3-data/clean/panel_train.pkl'
input_fp = '../3-data/clean/panel_btceth.pkl'
df = pd.read_pickle(input_fp)
lhs_col = 'r_tplus1'

# Clean up the data
df = df.set_index('date')
df = df.drop(['covar_mcap_t', 'covar_supply_adr_bal_100k_usd_t',
              'covar_supply_outside_ex_t'], axis=1)

# create new lhs variables
df = createPlatonicPortfolioWeights(df, min_weight=0.01, lhs_col=lhs_col)

# convert to single obseration per date
df = df.sort_values(by=['date', 'asset'])   
df = flattenToOneObPerDate(df, lhs_col)

In [ ]:
# TRY WITH AND WITHOUT NORMALIZATION
rhs_cols = list(df.columns.values)[4:]
for col in rhs_cols:
    df[col] = (df[col]-np.mean(df[col]))/np.std(df[col])

In [38]:
df = df.astype('float32')

In [39]:
df = df[df.index<np.max(df.index)]

In [ ]:
results_list = runCV(df, asset_universe_dict=None, lhs_col=lhs_col,
                     last_train_year=2019, val_end_year=2021,
                     arch_name='ffnn_one_ob_per_day')


In [329]:
results_list_master = results_list.copy()

In [25]:
# SAVE RESULTS
with open('../3-data/derived/validation_2020_2021_results_btceth_normalized.pickle', 'wb') as handle:
    pickle.dump(results_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [40]:

with open('../3-data/derived/validation_2020_2021_results_btceth_normalized.pickle', 'rb') as handle:
    results_list = pickle.load(handle)


In [41]:
# THIS WAS NOT NORMALIZED
# with open('../3-data/derived/validation_2020_2021_results_btceth.pickle', 'wb') as handle:
#     pickle.dump(results_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

# THIS WAS WITH OLD VARIABLES
# with open('../3-data/derived/validation_2018_2020_results_btceth.pickle', 'rb') as handle:
#     results_list = pickle.load(handle)

In [ ]:
# form validation period data
all_cols      = list(df.columns.values)
r_tplus_cols = [col for col in all_cols if col[:8] == lhs_col]
df = df[r_tplus_cols]
df = df[(df.index.year ==2020) | (df.index.year ==2021)]

# form validation period benchmark results
actual_simple_rets        = np.mean(df.values, axis=1)
actual_daily_simple_ret  = np.prod(1+actual_simple_rets)**(1/len(actual_simple_rets))-1
actual_annual_simple_ret  = (actual_daily_simple_ret+1)**(365.25)-1
actual_annual_sharpe      = np.mean(actual_simple_rets)/np.std(actual_simple_rets)*np.sqrt(365.25)
print('annual equal-weighted simple return: ' + 
      str(np.round(actual_annual_simple_ret, 4)))
print('annual equal-weighted sharpe: ' + 
      str(np.round(actual_annual_sharpe, 2)))
print('\n')
actual_simple_rets        = df['r_tplus1_bitcoin'].values
actual_daily_simple_ret   = np.prod(1+actual_simple_rets)**(1/len(actual_simple_rets))-1
actual_annual_simple_ret  = (actual_daily_simple_ret+1)**(365.25)-1
actual_annual_sharpe      = np.mean(actual_simple_rets)/np.std(actual_simple_rets)*np.sqrt(365.25)
print('BTC annual simple return: ' + 
      str(np.round(actual_annual_simple_ret, 4)))
print('BTC annual sharpe: ' + 
      str(np.round(actual_annual_sharpe, 2)))
print('\n')
actual_simple_rets        = df['r_tplus1_ethereum'].values
actual_daily_simple_ret   = np.prod(1+actual_simple_rets)**(1/len(actual_simple_rets))-1
actual_annual_simple_ret  = (actual_daily_simple_ret+1)**(365.25)-1
actual_annual_sharpe      = np.mean(actual_simple_rets)/np.std(actual_simple_rets)*np.sqrt(365.25)
print('ETH annual simple return: ' + 
      str(np.round(actual_annual_simple_ret, 4)))
print('ETH annual sharpe: ' + 
      str(np.round(actual_annual_sharpe, 2)))
print('\n')


In [43]:
# form validation period results
df = pd.DataFrame(data={'date': np.repeat(df.index.values, df.shape[1]),
                        'asset': np.tile([col[9:] for col in df.columns], df.shape[0]),
                        lhs_col: df.values.reshape(-1),
                        'prtfl_wght': np.vstack(results_list[0]['results_df'].yhats.values).reshape(-1)})
df = df.set_index('date')

In [44]:

long = 0.9
short = -0.1
long_threshold = 0
short_threshold = 0

In [45]:
df.loc[df.prtfl_wght>long_threshold, 'prtfl_wght'] = long
df.loc[df.prtfl_wght<short_threshold, 'prtfl_wght'] = short

annl_tc = calcAnnualTransactionCosts(df)
df['r'] = df.prtfl_wght*df[lhs_col]
r_df    = df.groupby(['date'])[['r']].sum()
annl_ret = calcPortfolioReturn(r_df)
annl_sharpe = calcPortfolioSharpe(r_df)
max_dd = max_draw_down(r_df)
max_1week_loss = max_1_week_loss(r_df)
print('annual transaction costs in simple return terms: ' + str(np.round(annl_tc, 4)))
print('annual simple return before trans costs: ' + str(np.round(annl_ret, 4)))
print('annual sharpe: '+str(np.round(annl_sharpe, 2)))
print('max drawdown : '+str(np.round(max_dd, 2)))
print('max one week loss : '+str(np.round(max_1week_loss, 2)))
print('LFG!!!!!')
print('\n')

annual transaction costs in simple return terms: -0.0513
annual simple return before trans costs: 6.2743
annual sharpe: 2.24
max drawdown : -0.6
max one week loss : -0.46
LFG!!!!!




In [62]:
# import and clean up data for test fitting
input_fp = '../3-data/clean/panel_btceth.pkl'
df = pd.read_pickle(input_fp)
df = df.set_index('date')
df = df.drop(['covar_mcap_t', 'covar_supply_adr_bal_100k_usd_t',
              'covar_supply_outside_ex_t'], axis=1)
df = createPlatonicPortfolioWeights(df, min_weight=0.01, lhs_col=lhs_col)
df = df.sort_values(by=['date', 'asset'])   
df = flattenToOneObPerDate(df, lhs_col)
rhs_cols = list(df.columns.values)[4:]
for col in rhs_cols:
    df[col] = (df[col]-np.mean(df[col]))/np.std(df[col])
df = df.astype('float32')
opt_hps = results_list[0].copy()
del opt_hps['results_df']
del opt_hps['end_date']
del opt_hps['mse']
del opt_hps['val_ret_delta']

# TEMP DROP OF LAST DATE
df = df[df.index < np.max(df.index)]

In [ ]:

# fit test period yhats
results_test_df = GenTestYhats(df, lhs_col, opt_hps) 


In [63]:
# form test period data
all_cols      = list(df.columns.values)
r_tplus_cols = [col for col in all_cols if col[:8] == lhs_col]
df = df[r_tplus_cols]
df            = df[df.index.year == 2022]


In [ ]:

# form test period benchmark results
actual_simple_rets        = np.mean(df.values, axis=1)
actual_weekly_simple_ret  = np.prod(1+actual_simple_rets)**(1/len(actual_simple_rets))-1
actual_annual_simple_ret  = (actual_weekly_simple_ret+1)**(365.25)-1
actual_annual_sharpe      = np.mean(actual_simple_rets)/np.std(actual_simple_rets)*np.sqrt(365.25)
print('annual equal-weighted simple return: ' + 
      str(np.round(actual_annual_simple_ret, 4)))
print('annual equal-weighted sharpe: ' + 
      str(np.round(actual_annual_sharpe, 2)))
print('\n')



In [ ]:

# form test period results
test_df      = pd.DataFrame(data={'date': np.repeat(df.index.values, df.shape[1]),
                                  'asset': np.tile([col[9:] for col in df.columns], df.shape[0]),
                                  lhs_col: df.values.reshape(-1),
                                  'prtfl_wght': np.vstack(results_test_df.yhats.values).reshape(-1)})
test_df      = test_df.set_index('date')
long = 0.5
short = -0.4
long_threshold = 0.5
short_threshold = -0.4
test_df.loc[test_df.prtfl_wght>long_threshold, 'prtfl_wght'] = long
test_df.loc[test_df.prtfl_wght<short_threshold, 'prtfl_wght'] = short

annl_tc      = calcAnnualTransactionCosts(test_df)
test_df['r'] = test_df.prtfl_wght*test_df[lhs_col]
r_df         = test_df.groupby(['date'])[['r']].sum()
annl_ret     = calcPortfolioReturn(r_df)
annl_sharpe  = calcPortfolioSharpe(r_df)
max_dd       = max_draw_down(r_df)
max_1mo_loss = max_1_week_loss(r_df)
print('annual transaction costs in simple return terms: ' + str(np.round(annl_tc, 4)))
print('annual simple return before trans costs: ' + str(np.round(annl_ret, 4)))
print('annual sharpe: '+str(np.round(annl_sharpe, 2)))
print('max drawdown : '+str(np.round(max_dd, 2)))
print('max one week loss : '+str(np.round(max_1mo_loss, 2)))

In [409]:
# TODO: SAVE THESE RESULTS TO SOME FILEAS BEAUT!


In [20]:
# TEMP FILE FOR MODEL HPs BUT I SHOULD USE THE FINAL TEST ONE
with open('../3-data/derived/validation_2020_2021_results_btceth_normalized.pickle', 'rb') as handle:
    results_list = pickle.load(handle)


In [21]:
# import and clean up data for quant strat
lhs_col = 'r_tplus1'
input_fp = '../3-data/clean/panel_btceth.pkl'
df = pd.read_pickle(input_fp)
df = df.set_index('date')

df = createPlatonicPortfolioWeights(df, min_weight=0.01, lhs_col=lhs_col)
df = df.sort_values(by=['date', 'asset'])   
df = flattenToOneObPerDate(df, lhs_col)
rhs_cols = list(df.columns.values)[4:]

for col in rhs_cols:
    df[col] = (df[col]-np.mean(df[col]))/np.std(df[col])
df = df.astype('float32')
opt_hps = results_list[0].copy()
del opt_hps['results_df']
del opt_hps['end_date']
del opt_hps['mse']
del opt_hps['val_ret_delta']

train_df = df[df.index < np.max(df.index)]
oos_df = df[df.index == np.max(df.index)]


In [ ]:
models = fitNN(train_df, opt_hps, lhs_col)

In [ ]:
oos_results_df = genYhats(oos_df, models, lhs_col)


In [24]:
# READ IN MASTER DATA
master_oos_results_df = pd.read_pickle('../3-data/clean/oos_yhats_2022.pickle')

In [25]:
# Append data
oos_results_df = pd.concat((master_oos_results_df, oos_results_df))

In [26]:
# SAVE NEW MASTER DATA
oos_results_df.to_pickle('../3-data/clean/oos_yhats_2022.pickle')

In [ ]:
oos_results_df

In [ ]:
pd.DataFrame(data=np.vstack(results_list[0]['results_df'].yhats.values)).describe()